In [73]:
import warnings
warnings.filterwarnings('ignore')

# numerical libraries
import numpy as np
import pandas as pd

# divide train and test (preproc)
from sklearn.cross_validation import train_test_split

# import different models
from sklearn.linear_model import LogisticRegression

# feature optimisation
from sklearn.feature_selection import SelectFromModel

# model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn import cross_validation

# standarisation of features
from sklearn import preprocessing

In [74]:
cd Dropbox/Portfolio/DataScience-Portfolio/Titanic

[Errno 2] No such file or directory: 'Dropbox/Portfolio/DataScience-Portfolio/Titanic'
/Users/Capgemini/Dropbox/Portfolio/DataScience-Portfolio/Titanic


In [75]:
# load data and test set
titanic = pd.read_csv('titanic_train_ready3.csv')

# generate X and Y for preditions
Y = np.ravel(titanic.Survived)  # to flatten array
X = titanic.drop('Survived', axis = 1)

In [76]:
# separate train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((623, 30), (268, 30), (623,), (268,))

In [77]:
# baseline accuracy (predicting that it did not survive)
baseline = round(1-np.mean(Y), 2)
baseline

0.62

In [78]:
# multi-colinearity
# make correlation matrix and select those with correlation above 0.7
corr_matrix = titanic.corr()
corr_matrix[corr_matrix > 0.7] = 1
corr_matrix[corr_matrix < -0.7] = 1
corr_matrix[corr_matrix != 1] = 0
corr_matrix

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Unknown,Age_mean,Minor,...,E,F,G,Unknown,Master,Miss,Mr,Mrs,Other,low_fare
Survived,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pclass,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Sex,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Age,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
SibSp,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Parch,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Fare,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age_Unknown,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Age_mean,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Minor,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [79]:
# Exploration to find those variables correlated above 0.7
coMat = pd.DataFrame(corr_matrix)
coMat.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Age_Unknown', 'Age_mean', 'Minor', 'Child', 'Alone', 'man_w_spouse',
       'woman_w_child', 'C', 'Q', 'S', 'A', 'B', 'C.1', 'D', 'E', 'F', 'G',
       'Unknown', 'Master', 'Miss', 'Mr', 'Mrs', 'Other', 'low_fare'],
      dtype='object')

In [80]:
coMat[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Age_Unknown', 'Age_mean', 'Minor', 'Child', 'Alone',]]

,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Unknown,Age_mean,Minor,Child,Alone
Survived,0,0,0,0,0,0,0,0,0,0,0
Pclass,1,0,0,0,0,0,0,0,0,0,0
Sex,0,1,0,0,0,0,0,0,0,0,0
Age,0,0,1,0,0,0,0,1,0,0,0
SibSp,0,0,0,1,0,0,0,0,0,0,0
Parch,0,0,0,0,1,0,0,0,0,0,0
Fare,0,0,0,0,0,1,0,0,0,0,0
Age_Unknown,0,0,0,0,0,0,1,0,0,0,0
Age_mean,0,0,1,0,0,0,0,1,0,0,0
Minor,0,0,0,0,0,0,0,0,1,1,0


In [81]:
coMat[['man_w_spouse',
       'woman_w_child', 'C', 'Q', 'S', 'A', 'B', 'C.1', 'D', 'E', 'F', 'G',]]

,man_w_spouse,woman_w_child,C,Q,S,A,B,C.1,D,E,F,G
Survived,0,0,0,0,0,0,0,0,0,0,0,0
Pclass,0,0,0,0,0,0,0,0,0,0,0,0
Sex,0,0,0,0,0,0,0,0,0,0,0,0
Age,0,0,0,0,0,0,0,0,0,0,0,0
SibSp,0,0,0,0,0,0,0,0,0,0,0,0
Parch,0,0,0,0,0,0,0,0,0,0,0,0
Fare,0,0,0,0,0,0,0,0,0,0,0,0
Age_Unknown,0,0,0,0,0,0,0,0,0,0,0,0
Age_mean,0,0,0,0,0,0,0,0,0,0,0,0
Minor,0,0,0,0,0,0,0,0,0,0,0,0


In [82]:
coMat[['Unknown', 'Master', 'Miss', 'Mr', 'Mrs', 'Other', 'low_fare']]

,Unknown,Master,Miss,Mr,Mrs,Other,low_fare
Survived,0,0,0,0,0,0,0
Pclass,1,0,0,0,0,0,0
Sex,0,0,0,1,0,0,0
Age,0,0,0,0,0,0,0
SibSp,0,0,0,0,0,0,0
Parch,0,0,0,0,0,0,0
Fare,0,0,0,0,0,0,0
Age_Unknown,0,0,0,0,0,0,0
Age_mean,0,0,0,0,0,0,0
Minor,0,0,0,0,0,0,0


In [83]:
# Drop following columns as they are hihgly correlated to other columns
X_train.drop(['Mr', 'Master', 'Unknown', 'Child', 'Age_mean'], axis = 1, inplace = True)
X_train.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_Unknown',
       'Minor', 'Alone', 'man_w_spouse', 'woman_w_child', 'C', 'Q', 'S', 'A',
       'B', 'C.1', 'D', 'E', 'F', 'G', 'Miss', 'Mrs', 'Other', 'low_fare'],
      dtype='object')

In [84]:
X_test.drop(['Mr', 'Master', 'Unknown', 'Child', 'Age_mean'], axis = 1, inplace = True)

### Logistic regression including all features

In [85]:
# make the model object and fit to training set
logReg = LogisticRegression()  
logReg.fit(X_train, Y_train)

# predict survivors on test set
predicted = logReg.predict(X_test)         
metrics.accuracy_score(Y_test, predicted)

0.79477611940298509

In [92]:
# cross validation
scores = cross_validation.cross_val_score(logReg, X_train, Y_train, cv=5)
scores.mean()

0.81715535074244749

In [87]:
# feature scaling
def standarisation(train, test):
    scaler = preprocessing.StandardScaler().fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)
    return train, test

In [93]:
# standarization of features
X_train_std, X_test_std = standarisation(X_train, X_test)

# Logistic Regression Model
logReg_std = LogisticRegression()  
logReg_std.fit(X_train_std, Y_train)   

# predict on test set
predicted = logReg_Std.predict(X_test_std) 
metrics.accuracy_score(Y_test,predicted)

0.79850746268656714

In [94]:
# cross validation
scores = cross_validation.cross_val_score(logReg_std, X_train_std, Y_train, cv=5)
scores.mean()

0.82038115719406046

### Recursive feature elimination

In [95]:
# Recursive Feature Selection (RFS)
logReg_RFS = SelectFromModel(logReg, prefit=True)
X_train_RFS = logReg_RFS.transform(X_train)
X_train_RFS.shape

(623, 9)

In [96]:
# which features were selected?
non_removedFeatures = logReg_RFS.inverse_transform(X_train_RFS)
non_removedFeatures = non_removedFeatures.sum(axis = 0)
non_removedFeatures[non_removedFeatures > 0 ] = 1

# make a dataframe with the features and whether they were selected (1)
coef_df = pd.DataFrame(X_train.columns)
coef_df.columns = ['Features']

coef_df['Coefficients'] = pd.Series(non_removedFeatures)
coef_df

,Features,Coefficients
0,Pclass,1
1,Sex,1
2,Age,0
3,SibSp,0
4,Parch,0
5,Fare,0
6,Age_Unknown,0
7,Minor,1
8,Alone,1
9,man_w_spouse,1


In [97]:
# select same features in test set
X_test_RFS = logReg_RFS.transform(X_test)
X_test_RFS.shape

(268, 9)

In [98]:
# re fit the logistic regression with the 2 features only
logReg_RFS = LogisticRegression()
logReg_RFS.fit(X_train_RFS, Y_train)

predicted = logReg_RFS.predict(X_test_RFS)  # predict affairs on test set
metrics.accuracy_score(Y_test,predicted)    # check accuracy

0.74626865671641796

In [99]:
# cross validation
scores = cross_validation.cross_val_score(logReg_RFS, X_train_RFS, Y_train, cv=5)
scores.mean()

0.79791623143881207

**Feature Scaling**

In [104]:
# Logistic Regression Model
X_train_RFS_std, X_test_RFS_std = standarisation(X_train_RFS, X_test_RFS)

logReg_RFS_std = LogisticRegression()  # make the model object
logReg_RFS_std.fit(X_train_RFS_std, Y_train)   # fit to the training set

predicted = logReg_RFS_std.predict(X_test_RFS_std)
metrics.accuracy_score(Y_test,predicted)

0.75

In [106]:
# cross validation
scores = cross_validation.cross_val_score(logReg_RFS_std, X_train_RFS_std, Y_train, cv=5)
scores.mean()

0.79790332821300558

### New model with 3 features only

In [107]:
# re fit the logistic regression with the 2 features only
logReg_3F = LogisticRegression()
logReg_3F.fit(X_train[['Pclass', 'Sex', 'Age']], Y_train)

predicted = logReg_3F.predict(X_test[['Pclass', 'Sex', 'Age']])  # predict affairs on test set
metrics.accuracy_score(Y_test,predicted)

0.77238805970149249

In [108]:
# cross validation
scores = cross_validation.cross_val_score(logReg_3F, X_train[['Pclass', 'Sex', 'Age']], Y_train, cv=5)
scores.mean()

0.79951623143881212

In [113]:
# with scaled features
X_train_3F_std, X_test_3F_std = standarisation(X_train[['Pclass', 'Sex', 'Age']], X_test[['Pclass', 'Sex', 'Age']])

logReg_3F_std = LogisticRegression()
logReg_3F_std.fit(X_train_3F_std, Y_train)

predicted = logReg_3F.predict(X_test_3F_std)  # predict affairs on test set
metrics.accuracy_score(Y_test,predicted)

0.42910447761194032

In [114]:
# cross validation
scores = cross_validation.cross_val_score(logReg_3F_std, X_train_3F_std, Y_train, cv=5)
scores.mean()

0.79949062980030727

# Prepare submission

In [115]:
titanic_test = pd.read_csv('titanic_test_ready3.csv')
titanic_test.drop(['Mr', 'Master', 'Unknown', 'Child', 'Age_mean'], axis = 1, inplace = True)
titanic_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Unknown,Minor,Alone,man_w_spouse,...,B,C.1,D,E,F,G,Miss,Mrs,Other,low_fare
0,3,1,34.5,0,0,7.8292,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,47.0,1,0,7.0000,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
2,2,1,62.0,0,0,9.6875,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,3,1,27.0,0,0,8.6625,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,3,0,22.0,1,1,12.2875,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1


In [117]:
titanic_train = X
titanic_train.drop(['Mr', 'Master', 'Unknown', 'Child', 'Age_mean'], axis = 1, inplace = True)
titanic_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Age_Unknown,Minor,Alone,man_w_spouse,...,B,C.1,D,E,F,G,Miss,Mrs,Other,low_fare
0,3,1,22,1,0,7.2500,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,0,38,1,0,71.2833,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,3,0,26,0,0,7.9250,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
3,1,0,35,1,0,53.1000,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4,3,1,35,0,0,8.0500,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [118]:
# standarise the datasets
titanic_train, titanic_test = standarisation(titanic_train, titanic_test)

In [119]:
# Logistic Regression Model
logReg_Final = LogisticRegression()  # make the model object
logReg_Final.fit(titanic_train, Y)   # fit to the training set

# cross validation
scores = cross_validation.cross_val_score(logReg_Final, titanic_train, Y, cv=5)
scores.mean()

0.81037415237392041

In [120]:
titanic_train.shape, titanic_test.shape

((891, 25), (418, 25))

In [121]:
predicted = logReg_Final.predict(titanic_test)

In [122]:
test_df = pd.read_csv('test.csv')

submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": predicted
    })

submission.to_csv('titanic_submission.csv', index=False)